## MERGING

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Cargar los archivos CSV como DataFrames
df_pop_filtered = pd.read_csv('format_data/df_pop_filtered.csv', delimiter=',', thousands='.', decimal=',')
df_gasstations_filtered = pd.read_csv('format_data/df_gasstations_filtered.csv', delimiter=',', thousands='.', decimal=',')
df_roads_filtered = pd.read_csv('format_data/df_roads_filtered.csv', delimiter=',', thousands='.', decimal=',')
df_accs_filtered = pd.read_csv('format_data/df_accs_filtered.csv', delimiter=',', thousands='.', decimal=',')
df_acc_vic_rec_filtered = pd.read_csv('format_data/df_acc_vic_rec_filtered.csv', delimiter=',', thousands='.', decimal=',')
df_vvmm_rce_filtered = pd.read_csv('format_data/df_vvmm_rce_filtered.csv', delimiter=',', thousands='.', decimal=',')

### Merging population and accs

In [3]:
# Merge the population and accidents dataframes on year, region, and province
df_pop_accs = pd.merge(df_pop_filtered, df_accs_filtered, on=['year', 'region', 'province'])

print(df_pop_accs.head())

   year              region  province  population  acc with victims  \
0  2013          País Vasco     Álava      202969               906   
1  2013  Castilla-La Mancha  Albacete      246555               481   
2  2013       C. Valenciana  Alicante     1194721              2465   
3  2013           Andalucía   Almería      448850               788   
4  2013     Castilla y León     Ávila       99333               378   

   total injuried  mortals  hosp injuries  minor injuries  
0            1216       17             86            1113  
1             680       19             87             574  
2            3530       42            341            3147  
3            1321       30            121            1170  
4             558       19             71             468  


### Giving some format to gas_stations to get only usable data

In [4]:
df_gasstations_filtered['gas_stations'] = df_gasstations_filtered[['95', '98', 'A', 'B']].max(axis=1)
print(df_gasstations_filtered.head())

   year     region province   95   98    A    B  gas_stations
0  2013  Andalucía  Almería  152  107  162   66           162
1  2013  Andalucía    Cádiz  178  143  190   49           190
2  2013  Andalucía  Córdoba  162   97  207  128           207
3  2013  Andalucía  Granada  211  164  226   77           226
4  2013  Andalucía   Huelva  106   66  120   53           120


In [5]:
# Merge df_pop_accs with df_gasstations_filtered to add the gas_stations column
df_pop_accs_gas = pd.merge(df_pop_accs, df_gasstations_filtered[['year', 'region', 'province', 'gas_stations']], on=['year', 'region', 'province'], how='left')

print(df_pop_accs_gas.head())

   year              region  province  population  acc with victims  \
0  2013          País Vasco     Álava      202969               906   
1  2013  Castilla-La Mancha  Albacete      246555               481   
2  2013       C. Valenciana  Alicante     1194721              2465   
3  2013           Andalucía   Almería      448850               788   
4  2013     Castilla y León     Ávila       99333               378   

   total injuried  mortals  hosp injuries  minor injuries  gas_stations  
0            1216       17             86            1113            56  
1             680       19             87             574           144  
2            3530       42            341            3147           344  
3            1321       30            121            1170           162  
4             558       19             71             468            55  


## ROADS MERGING

In [6]:
df_roads_filtered

,year,region,province,owner,free_motorways,multiple_road,single_road,toll_motorways
0,2013,Andalucía,Almería,CCAA,101.57,2.32,0.00,0.00
1,2013,Andalucía,Almería,CCAA,0.00,0.00,713.76,0.00
2,2013,Andalucía,Almería,State,203.50,1.12,0.00,28.19
3,2013,Andalucía,Almería,State,0.00,0.00,140.06,0.00
4,2013,Andalucía,Almería,provincial,0.00,0.00,1172.88,0.00
...,...,...,...,...,...,...,...,...
2867,2022,País Vasco,Álava,provincial,0.00,0.00,1273.66,0.00
2868,2022,La Rioja,"Rioja, La",CCAA,0.00,3.00,0.00,0.00
2869,2022,La Rioja,"Rioja, La",CCAA,0.00,0.00,1444.00,0.00
2870,2022,La Rioja,"Rioja, La",State,62.23,1.83,0.00,119.43


In [7]:
# Usar pivot_table para reestructurar el dataframe
df_reshaped = df_roads_filtered.pivot_table(
    index=['year', 'region', 'province'],
    columns='owner',
    values=['free_motorways', 'multiple_road', 'single_road', 'toll_motorways'],
    aggfunc='sum'
)

# Aplanar el MultiIndex de las columnas
df_reshaped.columns = [f'{road_type}_{owner}' for road_type, owner in df_reshaped.columns]

# Resetear el índice para que year, region y province sean columnas normales
df_reshaped.reset_index(inplace=True)

#Renombrar dataframe
df_roads_rdy = df_reshaped

# Mostrar el dataframe reestructurado
print(df_roads_rdy)

     year      region   province  free_motorways_CCAA  free_motorways_State  \
0    2013   Andalucía    Almería               101.57                203.50   
1    2013   Andalucía      Cádiz               138.09                116.88   
2    2013   Andalucía    Córdoba                 9.97                173.62   
3    2013   Andalucía    Granada               235.59                132.08   
4    2013   Andalucía     Huelva                32.96                125.65   
..    ...         ...        ...                  ...                   ...   
515  2022      Murcia     Murcia               167.00                360.42   
516  2022     Navarra    Navarra               236.81                  0.00   
517  2022  País Vasco  Guipúzcoa                  NaN                  0.00   
518  2022  País Vasco    Vizcaya                  NaN                  0.00   
519  2022  País Vasco      Álava                  NaN                  0.00   

     free_motorways_provincial  multiple_road_CCAA 

In [8]:
# Seleccionar las columnas de df_reshaped excepto year, region y province
columns_to_merge = df_roads_rdy.columns.difference(['year', 'region', 'province'])

# Unir los dataframes
df_final = pd.merge(df_pop_accs_gas, df_roads_rdy[['year', 'region', 'province'] + list(columns_to_merge)], on=['year', 'region', 'province'], how='left')

print(df_final.head())

   year              region  province  population  acc with victims  \
0  2013          País Vasco     Álava      202969               906   
1  2013  Castilla-La Mancha  Albacete      246555               481   
2  2013       C. Valenciana  Alicante     1194721              2465   
3  2013           Andalucía   Almería      448850               788   
4  2013     Castilla y León     Ávila       99333               378   

   total injuried  mortals  hosp injuries  minor injuries  gas_stations  ...  \
0            1216       17             86            1113            56  ...   
1             680       19             87             574           144  ...   
2            3530       42            341            3147           344  ...   
3            1321       30            121            1170           162  ...   
4             558       19             71             468            55  ...   

   free_motorways_provincial  multiple_road_CCAA  multiple_road_State  \
0                  

In [9]:
df_final

,year,region,province,population,acc with victims,total injuried,mortals,hosp injuries,minor injuries,gas_stations,...,free_motorways_provincial,multiple_road_CCAA,multiple_road_State,multiple_road_provincial,single_road_CCAA,single_road_State,single_road_provincial,toll_motorways_CCAA,toll_motorways_State,toll_motorways_provincial
0,2013,País Vasco,Álava,202969,906,1216,17,86,1113,56,...,92.27,NaN,0.00,11.96,NaN,0.00,1283.98,NaN,60.89,14.23
1,2013,Castilla-La Mancha,Albacete,246555,481,680,19,87,574,144,...,0.00,0.00,0.00,0.00,1596.85,420.40,1408.79,0.00,0.00,0.00
2,2013,C. Valenciana,Alicante,1194721,2465,3530,42,341,3147,344,...,0.00,58.40,65.29,16.08,906.90,208.49,996.38,0.00,159.46,0.00
3,2013,Andalucía,Almería,448850,788,1321,30,121,1170,162,...,0.00,2.32,1.12,0.00,713.76,140.06,1172.88,0.00,28.19,0.00
4,2013,Castilla y León,Ávila,99333,378,558,19,71,468,55,...,0.00,2.01,3.24,0.00,986.73,346.39,1065.00,0.00,37.40,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,2022,País Vasco,Vizcaya,680078,1883,2456,21,150,2285,127,...,104.51,NaN,0.00,39.62,NaN,0.00,1036.87,NaN,0.00,72.33
516,2022,Castilla y León,Zamora,92999,202,290,21,46,223,84,...,0.00,0.00,15.69,0.00,1136.91,422.94,1571.00,0.00,0.00,0.00
517,2022,Aragón,Zaragoza,576511,1589,2065,45,210,1810,249,...,0.00,9.41,36.11,3.15,2043.36,471.43,1016.75,5.28,54.91,0.00
518,2022,Ceuta,Ceuta,51147,257,366,1,10,355,9,...,NaN,0.00,1.04,NaN,21.02,15.19,NaN,0.00,0.00,NaN


In [10]:
df_final.to_csv('df_final.csv', index=False)